#### 텍스트 형식으로 된 데이터를 숫자 형식으로 변경하기

In [61]:
from openpyxl import load_workbook

wb = load_workbook("고객만족 데이터.xlsx") #다운받은 엑셀 파일을 wb 변수에 객체로 저장
ws = wb["Sheet1"] #시트 선택

#반복문을 통해 A2에서 F14에 있는 설문응답 데이터만 숫자형으로 변경
for row in ws:
    for col in row:
        if col.col_idx > 0 and col.col_idx < 7 and col.row > 1:
            col.data_type = "i"

#### 평균/합계 함수 입력

In [62]:
this_max_row = ws.max_row #max_row 멤버변수는 데이터가 입력된 마지막 행 번호를 출력해 줌
ws["A{}".format(this_max_row+1)] = "평균/합계" #마지막 행 첫 번째 열에 타이틀 입력
for col_let in ["B", "C", "D"]: #B, C, D열은 평균을 내는 열이므로 반복하며 동일한 엑셀 함수 입력
    average_formula = "=AVERAGE({}2:{}{})".format(col_let, col_let, this_max_row)
    ws["{}{}".format(col_let, this_max_row+1)] = average_formula
    ws["{}{}".format(col_let, this_max_row+1)].number_format = "#.##"

for col_let in ["E", "F"]: #E, F열은 합계를 내는 열이므로 반복하며 동일한 엑셀 함수 입력
    sum_formula = "=SUM({}2:{}{})".format(col_let, col_let, this_max_row)
    ws["{}{}".format(col_let, this_max_row+1)] = sum_formula

#### 새로운 시트를 추가해 집계표 만들기

In [63]:
wb.create_sheet("보고서 삽입용 집계표") #집계표가 들어갈 시트 생성
ws = wb["보고서 삽입용 집계표"] #방금 생성한 집계표 시트를 선택하여 ws 변수에 새롭게 저장

#집계표에 들어갈 내용을 리스트로 미리 지정 → 반복문을 통해 입력
summary_table_form = [
    ["성별 분포", "남성", "여성"],
    ["인원", "", ""],
    ["비율", "", ""],
    ["", "", ""],
    ["연령 분포", "20대", "30대", "40대", "50대 이상"],
    ["인원", "", ""],
    ["비율", "", ""],
    ["", "", ""],
    ["조사결과"],
    ["제품만족도"],
    ["재구매 의사"],
    ["추천의향"],
    ["방문횟수"],
    ["클레임 횟수"]
]
#리스트 항목마다 반복하며 append() 메서드 실행하여 집계표 내용 입력
for form_row in summary_table_form:
    ws.append(form_row)

In [64]:
for gender in [["B", "남"], ["C", "여"]]:
    ws[gender[0]+"2"] = "=COUNTIF(Sheet1!H2:H{}, \"{}\")".format(ws_max_row, gender[1])
    ws[gender[0]+"3"] = "={}2/SUM(B2:C2)".format(gender[0])
    ws[gender[0]+"3"].number_format = "0.00%"

for age in [["B", "20대"], ["C", "30대"], ["D", "40대"], ["E", "50대 이상"]]:
    ws[age[0]+"6"] = "=COUNTIF(Sheet1!G2:G{}, \"{}\")".format(ws_max_row, age[1])
    ws[age[0]+"7"] = "={}6/SUM(B6:E6)".format(age[0])
    ws[age[0]+"7"].number_format = "0.00%"

row_no = 10
for alphabet in ["B", "C", "D", "E", "F"]:
    ws["B"+str(row_no)] = "=Sheet1!{}{}".format(alphabet, ws_max_row+1)
    if row_no < 13:
        ws["B"+str(row_no)].number_format = "#.##"
    row_no += 1

#### 집계표 서식 추가하기

In [65]:
ws.column_dimensions["A"].width = 15
for alphabet in ["B", "C", "D", "E"]:
    ws.column_dimensions[alphabet].width = 10

for i in range(len(summary_table_form)+1):
    ws.row_dimensions[i].height = 25

In [66]:
from openpyxl.styles import Alignment, Side, Border, Font, PatternFill, Color

alignment_style = Alignment(horizontal= "center", vertical = "center")
side = Side(color="CCCCCC", border_style="medium")
border_style = Border(
    left = side,
    right = side,
    top = side,
    bottom = side,
)

style_apply_cells = ["A1:C3", "A5:E7", "A9:B14"]
for cell_area in style_apply_cells:
    for row in ws[cell_area]:
        for col in row:
            col.alignment = alignment_style
            col.border = border_style

In [67]:
header_font_style = Font(sz=12, bold=True)
header_background_style = PatternFill(fill_type="solid", start_color="ffd663")
style_apply_cells_for_header = ["A1:C1", "A5:E5", "A9:B9"]

for cell_area in style_apply_cells_for_header:
    for row in ws[cell_area]:
        for col in row:
            col.font = header_font_style
            col.fill = header_background_style

ws.merge_cells("A9:B9")

wb.active = ws
wb.save("고객만족 데이터_보고.xlsx")